In [ ]:
import pandas as pd
import numpy as np
import gresearch_crypto
import xgboost as xgb
import traceback
import datetime


TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

df = pd.read_csv(TRAIN_CSV)
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")

asset_to_weight = df_asset_details.Weight.values
df["Weight"] = df["Asset_ID"].apply(lambda x: asset_to_weight[x])

In [ ]:
def clean(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(how="any", inplace=True)

def test_train_split(df):
    X_train = df[df['timestamp'] <= 1623542400].drop('Target', axis=1)
    y_train = df[df['timestamp'] <= 1623542400].Target
    X_test = df[df['timestamp'] > 1623542400].iloc[:-1].drop('Target', axis=1)
    y_test = df[df['timestamp'] > 1623542400].iloc[:-1].Target
    return X_train, y_train, X_test, y_test

clean(df)
X_train, y_train, X_test, y_test = test_train_split(df)

In [ ]:
# NOT USED, I'll use it later though <3
from pandas import DataFrame
from pandas import concat
 
def time_lag(data, n_in=1, n_out=1, dropnan=True, interpolate = False):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    if interpolate:
        agg.fillna(method='bfill', inplace=True)
    return agg
 

In [ ]:
def process(df):
    df.loc[df["VWAP"] <= 0, "VWAP"] = df.loc[df["VWAP"] <= 0, "Close"]
    df["Unity"] = 1
    df["Open"] = np.log(df["Open"])
    df["High"] = np.log(df["High"])
    df["Low"] = np.log(df["Low"])
    df["Close"] = np.log(df["Close"])
    df["VWAP"] = np.log(df["VWAP"])
    #df['log_ret'] = np.log(df.Close/df.Open).fillna(0)
    
    #df['weird_feature'] = -(df['log_ret'] - (df['Weight'] * df['log_ret']).sum() / df['Weight'].sum())
    
    #norm_cols = ['Open','VWAP']
    #ref = "Close"
    #for col in norm_cols:
    #    df["norm_" + col] = df[col] / df[ref]
    
    #return df
    #return pd.concat([df, time_lag(df[["VWAP", "Volume", "Open", "Close"]], n_in=1, n_out=0, dropnan=False, interpolate=True)], axis=1)

#X_train = 
process(X_train)
#X_test = 
process(X_test)

In [ ]:
X_train.head()

In [ ]:
def f1(rows):
    return (rows["Weight"]*rows["Close"]).sum()/rows["Weight"].sum()
def f2(rows):
    return (rows["Weight"]*rows["Open"]).sum()/rows["Weight"].sum()

X_test["mClose"] = X_test.timestamp.map(X_test.groupby("timestamp").apply(f1))
X_test["mOpen"] = X_test.timestamp.map(X_test.groupby("timestamp").apply(f2))

In [ ]:
X_train["mClose"] = X_train.timestamp.map(X_train.groupby("timestamp").apply(f1))
X_train["mOpen"] = X_train.timestamp.map(X_train.groupby("timestamp").apply(f2))

In [ ]:
#(X_test["mClose"] - X_test["mOpen"]).head(30)

In [ ]:
# NOT USED
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

def generate_features(df, lag = 1, shuffle = False):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    
    #df_feat.fillna(-999,inplace=True)
    if lag > 0:
        df_feat = series_to_supervised(df_feat, n_in = lag)
    df_feat['Upper_Shadow'] = upper_shadow(df)
    df_feat['Lower_Shadow'] = lower_shadow(df)
    if shuffle is True:
        df_feat = df_feat.sample(frac=1)
    return df_feat

In [ ]:
X_train.columns

In [ ]:
from sklearn.preprocessing import StandardScaler


class BestModel:
    def __init__(self):
        self.beta = None
        self.scaler = StandardScaler()
    
    def fit(self, X_train, y_train):
        #self.scaler.fit(X_train)
        #X = self.scaler.transform(X_train)
        X = X_train.values
        k = 0.5
        mat = X.T@X
        mat += k*np.identity(len(mat))
        self.beta = np.linalg.inv(mat)@X.T@y_train.values
        
    def predict(self, X_test):
        #X = self.scaler.transform(X_test)
        X = X_test.values
        return X@self.beta

In [ ]:
features = ["Count", "Open", "Close", "High", "Low", "Volume", "VWAP", "Unity", "mClose", "mOpen"]#, 'weird_feature']
print(X_train.columns)

In [ ]:
models = [BestModel() for _ in range(len(df_asset_details))]

y_pred = pd.Series(data=np.full_like(y_test.values, np.nan), index=y_test.index)
for asset_ID, model in enumerate(models):
    X_asset_train = X_train[X_train.Asset_ID == asset_ID]
    y_asset_train = y_train[X_train.Asset_ID == asset_ID]
    X_asset_test = X_test[X_test.Asset_ID == asset_ID]
    
    model.fit(X_asset_train[features], y_asset_train)
    y_pred[X_test.Asset_ID == asset_ID] = model.predict(X_asset_test[features])
    print(f"Trained model for asset {asset_ID}")

In [ ]:
def corr(a, b, w):
    cov = lambda x, y: np.sum(w * (x - np.average(x, weights=w)) * (y - np.average(y, weights=w))) / np.sum(w)
    return cov(a, b) / np.sqrt(cov(a, a) * cov(b, b))

alt_weight = np.ones_like(y_pred)
R = corr(y_pred, y_test.values, X_test.Weight)
print(f"{R:.5f}")
# 0.01641 shenanigans
# 0.01523 remove two
#0.01464 all
# 0.01519 - log
#0.01784587806310679

#0.01992 - 0.01
#0.02151 - 0.5
# 0.02205 - 1.0
# 0.02235 - 2.0
#0.02081 - 0.2

In [ ]:
from scipy.stats import pearsonr
print(pearsonr(y_pred, y_test))

In [ ]:
for i in range(len(models[0].beta)):
    print(f"{features[i]}: {models[0].beta[i]/np.sum(np.abs(models[0].beta)):.8f}")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.abs(models[0].beta))
plt.ylim(bottom=0)

# Predict & submit

References: [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)

Something that helped me understand this iterator was adding a pdb checkpoint inside of the for loop:

```python
import pdb; pdb.set_trace()
```

See [Python Debugging With Pdb](https://realpython.com/python-debugging-pdb/) if you want to use it and you don't know how to.


In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    df_pred['Target'] = np.nan
    
    df_test["Weight"] = df_test["Asset_ID"].apply(lambda x: asset_to_weight[x])
    process(df_test)
    df_test["mClose"] = (df_test['Weight'] * df_test['Close']).sum() / df_test['Weight'].sum()
    df_test["mOpen"] = (df_test['Weight'] * df_test['Open']).sum() / df_test['Weight'].sum()
    
    for asset_ID, model in enumerate(models):
        X_asset_test = df_test[df_test.Asset_ID == asset_ID]
        df_pred.loc[df_test.Asset_ID == asset_ID, 'Target'] = model.predict(X_asset_test[features])
    df_pred['Target'] = df_pred['Target'].interpolate('nearest')
    env.predict(df_pred)

In [ ]:
print("oh yes!")